# **Milestone 3: Sources, Forms, and Quantification of Bias and Discrimination in Supervised Learning**
## **PRACTICE NOTEBOOK 3 - Evaluate model bias using existing libraries (aif360)**


In this part of the course, we will look for bias using a practical example. A  company is looking to hire a new employee. They use a machine learning algorithm to select the top candidates. The candidates are assigned either 0 if they're not selected or 1 if they are. 

There are 4 practice notebooks in total (current one in red):
1. Explore given data to: detect potential bias early & check for proxies 
2. Evaluate model bias "manually"
3. <font color='red'> **Evaluate model bias using existing libraries (aif360)**</font>
4. Example code to get confidence intervals for a metric (nothing to do)

Instructions to complete in each parts are in bold. Intermediate results are given so one can continue the exercise. 

This is notebook number 3. In the previous notebook, we measured bias manually. There exist a number of python library facilitating calculations for us. In this part, we will replicate the results found in the previous section using aif360. In this notebook, we will:
- Import data and useful modules
- Install and import aif360
- Learn how to create aif360 BinaryLabelDatasets object
- Learn how to calculate fairness metrics using aif360

We evaluate here the same metrics as in the previous notebook. You should get the same results as what you calculated "manually" in notebook 2. Here we only work with the "Black" unprivileged group. As a reminder, these are the results from the previous notebook that you should be able to reproduce here:

| Metric | Value | 
| --- | --- | 
| Statistical Parity |-0.09 | 
| Disparate Impact | 0.75 |
| Equal Opportunity Difference | -0.11 | 

## **0 - Import modules, load data and useful functions**

In [ ]:
#imports 
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier

In [ ]:
# Only run if running on Google Colab
!pip3 install pickle5
import pickle5 as pickle
!gdown --id 1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg3R # download data file from public link and place it in content/ folder

     |████████████████████████████████| 132 kB 8.8 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219327 sha256=d4d370f68d1319e495a1d41b12574618fcb576f83e9c6da0ed5aae2a41840a7a
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5
Downloading...
From: https://drive.google.com/uc?id=1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg3R
To: /content/data.pickle
128MB [00:01, 89.9MB/s]


In [ ]:
'''
Uncomment the code below if you are running this from your local machine
Note: Place data.pickle file in the same folder as the .ipynb notebook (Download link: https://docs.google.com/uc?export=download&id=1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg)
'''
# with open('data.pickle', 'rb') as handle:
#     raw_data = pickle.load(handle)   
if 'google.colab' in str(get_ipython()): # if running from colab
  with open('/content/data.pickle', 'rb') as handle:
      raw_data = pickle.load(handle)  
raw_data[:5] #display the first 5 candidates data

,Label,Gender,Ethnicity,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,0,Female,White,28.021737,4.351153,2.453895,1.637143,-1.746628,-0.483463,0.034170,1.399225,-0.795440,0.417474,0.214564,-0.471581,1.945645,-0.676217,1.213878,0.015701,1.472670,-0.054158,0.106858,-1.073194,-1.071848,-0.249942,0.634626,-0.732358,2.445728,0.784284,0.112329,1.055362,-0.605459,1.259140,-0.287927,0.214142,-0.644585,1.165376,-0.409198,-0.705823,0.091147,...,-0.655416,-0.012623,0.660826,0.258141,0.036875,-0.229340,0.353817,-0.178814,-0.145229,-0.040692,-0.046980,0.311939,-0.348202,0.271357,0.355443,-0.050447,-0.051816,0.083028,0.184139,0.107824,-0.083415,-0.359288,0.156547,-0.588539,-0.025777,-0.172269,0.331421,0.222768,-0.319124,-0.060476,-0.557444,-0.015627,-0.052749,-0.234189,-0.072384,0.090403,0.376761,0.258914,-0.050558,0.014513
1,0,Female,White,29.603342,-3.407193,0.771800,-2.957411,0.599226,-2.805277,0.329414,-2.055339,-1.194446,-0.633159,2.268302,1.159443,0.899266,-0.472739,0.541605,-1.248643,0.046512,1.225688,0.456477,-1.483071,-0.944882,1.483229,0.512809,0.692537,0.178988,-1.609531,-1.985852,-0.469491,-1.156583,0.475535,-0.041015,-0.214832,-0.681641,1.131433,-0.667814,0.267111,-0.112433,...,0.034605,-0.024487,-0.212205,0.440684,-0.065303,0.409878,0.455144,-0.108402,0.027760,-0.015238,0.027453,0.319960,-0.014589,-0.083241,-0.285702,0.047510,-0.144107,0.405289,-0.044139,-0.287215,0.201876,-0.298703,0.347969,0.029646,0.073052,-0.010259,0.023681,0.373202,-0.525402,-0.198727,-0.198440,-0.158843,0.191984,-0.004532,0.229210,-0.173042,-0.072871,0.442939,-0.054423,0.026959
2,1,Female,Hispanic,26.504283,0.642464,2.522944,-2.197094,2.270646,-0.472510,0.532815,-0.266449,-0.131638,1.038315,-0.865827,-0.811267,-0.381401,-0.801701,-0.485021,0.656005,2.489571,-0.714447,0.658228,-0.075957,-1.159888,-2.334786,-0.253364,-2.073697,-0.939994,-1.177166,0.551689,-1.313316,-0.486217,0.732130,-0.320456,-1.143053,1.297522,-0.617038,0.340978,0.978603,0.398515,...,0.510825,-0.616479,0.644675,0.505319,-0.299894,-0.058435,0.095024,-0.101136,0.042583,0.061005,0.304137,0.259210,-0.022425,0.138097,-0.442536,-0.108350,0.369865,0.151049,0.096285,0.013651,0.175281,0.144344,-0.006250,0.100850,-0.051642,0.122977,-0.088661,-0.229844,-0.272144,0.012633,0.423352,-0.033844,-0.125387,-0.483924,-0.116553,-0.113281,0.015519,0.017111,-0.012309,0.264572
3,0,Female,Hispanic,25.012088,0.895121,-2.092517,3.687830,0.539642,1.988930,1.121646,2.255337,-0.128801,1.148379,1.616247,-2.599757,-0.322807,2.102508,-0.204551,0.069818,0.745222,-0.859875,-2.235995,-0.207436,-1.678697,-0.569024,-0.723122,-0.144833,-1.537487,1.678429,0.501249,-0.230747,0.746559,-0.069959,-0.346651,0.448291,0.283592,-0.445759,-0.529080,0.287333,0.466766,...,-0.153845,-0.049137,-0.023112,-0.173509,-0.265363,0.091898,0.016743,-0.092894,-0.009915,-0.031731,0.153983,0.001281,0.123019,-0.035719,-0.045633,-0.103204,0.089567,0.104990,0.337228,-0.018783,-0.215437,0.268139,-0.125425,0.095183,-0.125172,-0.226467,0.371647,-0.023041,-0.093040,0.338300,-0.280392,0.046582,0.116709,0.133876,0.072716,0.124083,0.213735,-0.149901,-0.217130,0.004403
4,1,Male,Hispanic,27.358934,-2.332423,0.154999,-2.623793,1.682456,1.262280,-1.685565,0.489319,-0.043471,-0.372265,1.778535,-1.145419,2.461327,1.396318,-0.911969,-2.228570,1.378633,-1.512325,-0.440331,-0.111163,-0.885884,-0.840501,1.576620,-0.972075,-2.008346,-0.358732,0.896535,0.562193,0.154542,-1.077315,1.902062,1.728109,0.317205,-0.436143,0.226549,-0.502206,-0.157102,...,0.181831,-0.026589,-0.051453,0.261819,-0.048644,-0.099526,-0.026777,0.039836,-0.168277,0.077232,0.193722,0.093298,-0.075132,-0.063202,0.120167,0.039270,0.350429,0.166559,0.130134,-0.181019,-0.193276,0.312204,-0.187331,-0.029194,-0.212277,-0.463872,0.041810,0.041185,-0.182479,-0.182461,-0.019350,-0.093371,0.0034

In [ ]:
# remove all nans --> we use the variable "data" in the rest of this notebook
data = raw_data.dropna()

In [ ]:
def split_data_from_df(data):
  y = data['Label'].values
  # g = data['Gender'].values
  # e = data['Ethnicity'].values
  X = data[np.arange(500)].values
  filter_col = ['Ethnicity','Gender'] + [col for col in data if str(col).startswith('Ethnicity_')] + [col for col in data if str(col).startswith('Gender_')] 
  dem = data[filter_col].copy()
  return X,y,dem
def encode(df):
  g_enc = LabelEncoder()
  e_enc = LabelEncoder()
  df['Gender'] = g_enc.fit_transform(df['Gender'])
  df['Ethnicity'] = e_enc.fit_transform(df['Ethnicity'])
  return df, g_enc,e_enc

## **1. Install and import aif360**

In [ ]:
!pip install aif360 #install aif360

In [ ]:
import aif360
from aif360.metrics import ClassificationMetric
from aif360.datasets import BinaryLabelDataset

## **2. Create BinaryLabelDatasets objects**

We first need to process our data so it is in the standard aif360 format. aif360 uses custom dataset classes. We will use the BinaryLabelDataset for our problem. The BinaryLabelDataset has four components:
- Features : features used to train the model
- Labels : Outcome of binary classification
- Scores: Probability of outcome (not available in our case)
- Protected characteristics (*Gender* and *Ethnicity* columns for us)

In the code below we define the model and split our data into train and test set.

In [ ]:
model = RidgeClassifier(random_state = 42)
# split into train/test
data_train, data_test = train_test_split(data,test_size = 0.3,random_state=4)
# # drop unecessary columns
# cols_to_keep = ['Label','Gender','Ethnicity'] + list(np.arange(500))
# data_train = data_train[cols_to_keep].copy()
# data_test = data_test[cols_to_keep].copy()
# make Gender and Ethnicity numerical
data_train,g_enc,e_enc = encode(data_train.copy())
data_test,_,_ = encode(data_test.copy())
print("Gender codes - Male : %d - Female : %d"%(g_enc.transform(['Male'])[0],g_enc.transform(['Female'])[0]))
print("Ethnicity codes - White : %d - Black : %d - Asian : %d - Hispanic : %d"%(e_enc.transform(['White'])[0],e_enc.transform(['Black'])[0],e_enc.transform(['Asian'])[0],e_enc.transform(['Hispanic'])[0]))

Gender codes - Male : 1 - Female : 0
Ethnicity codes - White : 3 - Black : 1 - Asian : 0 - Hispanic : 2


**Questions** : 
- **Make two BinaryLabelDatasets: one for data_train and one for data_test.** See [documentation here](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.BinaryLabelDataset.html#aif360.datasets.BinaryLabelDataset) for BinaryLabelDataset and [here](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.StructuredDataset.html#aif360.datasets.StructuredDataset) for StructuredDataset the base class on which the former is built. You will need to specify the following arguments: *favorable_label, unfavorable_label,df,label_names,protected_attribute_names*.
- **Update the features so they don't contain the protected attributes.** By default, the protected attributes are used as features by the BinaryLabelDataset object. You can check which columns are included using *ds.feature_names*. You can then specify which features you want to use instead using *ds.feature_names = np.arange(500).astype(str)*. Here we set the features to be the 500 pre-defined features, whose columns names are 0,1..499.

## **3. Train the model and get the predictions for the test set**

- **Train the model and get the predictions for the test set.** You can use *X_train=ds_train.features* and *y_train=ds_train.labels.ravel()* to access X and y from the dataset you created earlier; and similarly for *X_test,y_test*.

## **4. Create a ClassificationMetric object and calculate accuracy and faitness metrics**

Now we will need to choose what privileged group and unprivileged group we want to focus on. We will demonstrate for *Black* vs *White*. We define our privileged group and unprivileged group as such:

```python
    unprivileged_groups = [{'Ethnicity':1}]  #1 is the code for 'Black'
    privileged_groups = [{'Ethnicity':3}]   #3 is the code for 'White 
```
The codes for each gender/ethnicity are given just after where we define the model and train/test set.
           
**Questions** : 
- **Create a ClassificationMetric object for the test set**. See [doc](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html). It takes 4 arguments: 
    1. dataset: ds_test = the test dataset you created above 
    2. classified_dataset: ds_test_pred = the same dataset but replacing the labels with the predictions as such:
    ```python
            ds_test_pred = ds_test.copy()
            ds_test_pred.labels = y_pred_test
    ```
    3. unprivileged_groups
    4. privileged_groups
    
- **Calculate the Accuracy**
- **Claculate the Statistical Parity Difference, Disparate Impact, Equal Opportunity Difference.** 
 The available metrics are [here](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html).
 
 
Note: if you specify more than one privileged/unprivileged groups, aif360 aggregates them as if they were one privileged/unprivileged group and compute the fairness metrics for the aggregate. Hence to compute for instance the fairness metrics for the *Asian* group, you should create a new ClassificationMetric object.

## **Final remarks**
**You should find the same results as in the previous section**. 

Note that for each analysis, we only obtained one number for fairness metrics. As mentioned before, in a real life setting, you should calculate a confidence interval by repeating the experiments for different train/test splits. A way to do this is for instance to use a non-parametric bootstrap method as described in section 10.2 from the book [Computer Age Statistical Inference by B.Efron and T.Hastie](https://web.stanford.edu/~hastie/CASI_files/PDF/casi.pdf) that we linked earlier. We give an example of this in the next and last notebook. Note that there will be nothing for you to do in this last notebook. 